In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime
import warnings
warnings.simplefilter("ignore")

# Store CSV into DataFrame

In [2]:
path_i="invoice_account_list.csv"
invoice_account=pd.read_csv(path_i)

In [3]:
path_l="lockerstock.xlsx"
lockerstock=pd.read_excel(path_l)

# Read all the file in target folder clean up data and store in Data Frame

In [4]:
loc="C:/Users/rwang/Desktop/ETL projec/Order Sheet/Current order sheet/"
os.chdir(loc)
data = []
files = [f for f in os.listdir(loc) if os.path.isfile(f)]
for f in files:      
    if "$" not in f:
        df=pd.read_excel(loc+f)
        new_header = df.iloc[2]
        df.columns = new_header
        df =pd.DataFrame(df[7:])
        df.set_index('AX STYLE ',inplace=True)
        df2=pd.DataFrame(df.stack())   
#         df2=df.melt(id_vars=['AX STYLE '], 
#         var_name="Item_name", 
#         value_name="Value")
        df2.dropna(inplace=True)  
        data.append(df2)   
exportdata=pd.concat(data)

In [5]:
exportdata.reset_index(inplace=True)
newexportdata=exportdata[exportdata['AX STYLE ']!="Total "]
newexportdata.rename(columns={"AX STYLE ":"store_name",2:"item_number",0:"qty"},inplace=True)
newexportdata

,store_name,item_number,qty
0,MARC JACOBS CHINA BEIJING SKP,X0000363-001,1
1,MARC JACOBS CHINA BEIJING SKP,X0000364-001,1
2,MARC JACOBS CHINA BEIJING SKP,X0000365-001,1
8,MJI FRANCE - PRINTEMPS THE - NON-STK ITEMS & B...,X0000363-001,1
9,MJI FRANCE - PRINTEMPS THE - NON-STK ITEMS & B...,X0000364-001,1
...,...,...,...
149,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000352-100,1
150,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000353-100,1
151,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000354-100,1
152,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000355-100,1


# Connect to local database

In [6]:
connection_string = "postgres:postgres @localhost:5432/pkgorder_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
engine.table_names()

['upload', 'invoice_list', 'lockerstock', 'order_summary', 'joindata_i']

# Load invoice_account list and lockerstock file into database

In [8]:
invoice_account.to_sql(name='invoice_list', con=engine, if_exists='replace', index=False)

In [9]:
lockerstock.to_sql(name='lockerstock', con=engine, if_exists='replace', index=False)

# Load combined order summary Data Frame into database

In [10]:
newexportdata.to_sql(name='order_summary', con=engine, if_exists='replace', index=False)

# Confirm data has been added by querying the invoice_list table

In [11]:
pd.read_sql_query('select * from invoice_list', con=engine).head()

,invoice_account,customer_name,customer_account,invoice_address,customer classification group,door,warehouse,price_catelog
0,923279,MARC JACOBS JAPAN RETAIL UNIFORM,4002001,Order account,Inter Japa,XX9999,13200,JPNWHL
1,4000012,MARC JACOBS INTERNATIONAL BOOKMARC NY UNIFORM,4002384,Order account,Admin,US1026,10100,USAWHL
2,4000012,MARC JACOBS INTERNATIONAL LOS ANGELES UNIFORM,4002381,Order account,Admin,US1008,10100,USAWHL
3,4000012,MARC JACOBS INTERNATIONAL NY MADISON UNIFORM,4002851,Order account,Admin,US1038,10100,USAWHL
4,4000012,MARC JACOBS INTERNATIONAL NYC GREENE STREET UN...,4002989,Order account,Admin,US1039,10100,USAWHL


# Confirm data has been added by querying the order_summary table

In [12]:
pd.read_sql_query('select * from order_summary', con=engine).head()

,store_name,item_number,qty
0,MARC JACOBS CHINA BEIJING SKP,X0000363-001,1
1,MARC JACOBS CHINA BEIJING SKP,X0000364-001,1
2,MARC JACOBS CHINA BEIJING SKP,X0000365-001,1
3,MJI FRANCE - PRINTEMPS THE - NON-STK ITEMS & B...,X0000363-001,1
4,MJI FRANCE - PRINTEMPS THE - NON-STK ITEMS & B...,X0000364-001,1


# Inner Join with invoice_account_list

In [13]:
joindata_i=pd.read_sql_query('SELECT order_summary.store_name,order_summary.item_number,order_summary.qty,invoice_list.customer_account,invoice_list.warehouse,invoice_list.price_catelog \
 FROM order_summary \
 JOIN invoice_list ON order_summary.store_name=invoice_list.customer_name ', con=engine)
joindata_i

,store_name,item_number,qty,customer_account,warehouse,price_catelog
0,MARC JACOBS CHINA BEIJING SKP,X0000363-001,1,4000913,13200,USAWHL
1,MARC JACOBS CHINA BEIJING SKP,X0000364-001,1,4000913,13200,USAWHL
2,MARC JACOBS CHINA BEIJING SKP,X0000365-001,1,4000913,13200,USAWHL
3,MJI FRANCE - PRINTEMPS THE - NON-STK ITEMS & B...,X0000363-001,1,4003045,12100,EURWHL
4,MJI FRANCE - PRINTEMPS THE - NON-STK ITEMS & B...,X0000364-001,1,4003045,12100,EURWHL
...,...,...,...,...,...,...
109,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000352-100,1,4002389,10100,USAWHL
110,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000353-100,1,4002389,10100,USAWHL
111,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000354-100,1,4002389,10100,USAWHL
112,MARC JACOBS INTERNATIONAL CABAZON UNIFORM,X0000355-100,1,4002389,10100,USAWHL


In [14]:
joindata_i.to_sql(name='joindata_i', con=engine, if_exists='replace', index=False)

# Perform Data Left Join on lockerstock table

In [15]:
joindata_l=pd.read_sql_query('SELECT joindata_i.customer_account,joindata_i.item_number,joindata_i.warehouse,joindata_i.price_catelog,joindata_i.qty,lockerstock.sales_order\
 FROM joindata_i \
 LEFT JOIN lockerstock ON joindata_i.store_name=lockerstock.customer_name AND joindata_i.warehouse=lockerstock.warehouse', con=engine)
joindata_l

,customer_account,item_number,warehouse,price_catelog,qty,sales_order
0,4000913,X0000363-001,13200,USAWHL,1,None
1,4000913,X0000364-001,13200,USAWHL,1,None
2,4000913,X0000365-001,13200,USAWHL,1,None
3,4003045,X0000363-001,12100,EURWHL,1,None
4,4003045,X0000364-001,12100,EURWHL,1,None
...,...,...,...,...,...,...
109,4002389,X0000352-100,10100,USAWHL,1,None
110,4002389,X0000353-100,10100,USAWHL,1,None
111,4002389,X0000354-100,10100,USAWHL,1,None
112,4002389,X0000355-100,10100,USAWHL,1,None


In [16]:
today = datetime.date.today()
joindata_l["ship_date"] = today.strftime("%m/%d/%Y")
joindata_l["cancel_date"]=(today+datetime.timedelta(days=90)).strftime("%m/%d/%Y")
joindata_l["order_category"]="OPX"
joindata_l["division"]="PKG"
joindata_l["size"]="1SZ"
joindata_l["salesid"]="DUMMY"
joindata_l

,customer_account,item_number,warehouse,price_catelog,qty,sales_order,ship_date,cancel_date,order_category,division,size,salesid
0,4000913,X0000363-001,13200,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
1,4000913,X0000364-001,13200,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
2,4000913,X0000365-001,13200,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
3,4003045,X0000363-001,12100,EURWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
4,4003045,X0000364-001,12100,EURWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
...,...,...,...,...,...,...,...,...,...,...,...,...
109,4002389,X0000352-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
110,4002389,X0000353-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
111,4002389,X0000354-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY
112,4002389,X0000355-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY


In [17]:
joindata_l["order_subtype"]=""
joindata_l["order_type"]=""
joindata_l["customer_po"]=""
for i in range(len(joindata_l)):
        if joindata_l["sales_order"][i] is None:
            joindata_l["order_subtype"][i]="Sales - Regular"
            joindata_l["order_type"][i]="Sales - Order"
            joindata_l["customer_po"]=joindata_l["order_category"].astype(str)+" "+joindata_l["ship_date"].astype(str)+" "
        else: 
            joindata_l["order_subtype"][i]="Sales - Release"
            joindata_l["order_type"][i]="Journal"
            joindata_l["customer_po"]=joindata_l["order_category"].astype(str)+" "+joindata_l["ship_date"].astype(str)+" "+joindata_l["sales_order"].astype(str)
joindata_l

,customer_account,item_number,warehouse,price_catelog,qty,sales_order,ship_date,cancel_date,order_category,division,size,salesid,order_subtype,order_type,customer_po
0,4000913,X0000363-001,13200,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
1,4000913,X0000364-001,13200,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
2,4000913,X0000365-001,13200,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
3,4003045,X0000363-001,12100,EURWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
4,4003045,X0000364-001,12100,EURWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,4002389,X0000352-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
110,4002389,X0000353-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
111,4002389,X0000354-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020
112,4002389,X0000355-100,10100,USAWHL,1,None,05/03/2020,08/01/2020,OPX,PKG,1SZ,DUMMY,Sales - Regular,Sales - Order,OPX 05/03/2020


In [18]:
uploaddata=joindata_l[["customer_account","price_catelog","order_category","division","customer_po","order_subtype","order_type","ship_date","cancel_date",'warehouse',"item_number",'size','qty','salesid',"sales_order"]]
uploaddata

,customer_account,price_catelog,order_category,division,customer_po,order_subtype,order_type,ship_date,cancel_date,warehouse,item_number,size,qty,salesid,sales_order
0,4000913,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,13200,X0000363-001,1SZ,1,DUMMY,None
1,4000913,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,13200,X0000364-001,1SZ,1,DUMMY,None
2,4000913,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,13200,X0000365-001,1SZ,1,DUMMY,None
3,4003045,EURWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,12100,X0000363-001,1SZ,1,DUMMY,None
4,4003045,EURWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,12100,X0000364-001,1SZ,1,DUMMY,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,4002389,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,10100,X0000352-100,1SZ,1,DUMMY,None
110,4002389,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,10100,X0000353-100,1SZ,1,DUMMY,None
111,4002389,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,10100,X0000354-100,1SZ,1,DUMMY,None
112,4002389,USAWHL,OPX,PKG,OPX 05/03/2020,Sales - Regular,Sales - Order,05/03/2020,08/01/2020,10100,X0000355-100,1SZ,1,DUMMY,None


# Load upload data in postgres database and export as csv

In [19]:
uploaddata.to_sql(name='upload', con=engine, if_exists='replace', index=False)

In [20]:
uploaddata.to_csv("../../upload.csv")